In [1]:
Нужно разобрать этот пост полностью, кажется это 80 % всего что нужно про абтесты знать

https://vkteam.medium.com/practitioners-guide-to-statistical-tests-ed2d580ef04f

SyntaxError: invalid syntax (<ipython-input-1-364e1fde4ac1>, line 1)

# Cuped